In [1]:
import pandas as pd
from sqlalchemy import create_engine
import yaml

In [2]:
# Read csv
trade_df = pd.read_csv("./data/trade.csv", encoding="utf-8", 
                       dtype={'HSCD': str, "QTY": float, 'BYRADDR2': str, 'SPLYADDR2': str})
# trade_df.head()

In [4]:
trade_df.columns

Index(['HSCD', 'QTY', 'QTYUNIT', 'AMT', 'CUR', 'USDUNTPRICE', 'UNTPRICE',
       'ITEM', 'BYRORGNM1', 'BYRPRTNUM', 'BYRADDR1', 'BYRADDR2', 'SPLYORGNM1',
       'SPLYPRTNUM', 'SPLYADDR1', 'SPLYADDR2', 'ISSUEDT', 'TOTQTY',
       'TOTQTYUNIT', 'TOTAMT', 'TOTAMTCUR'],
      dtype='object')

In [9]:
len(trade_df)

2492786

In [3]:
trade_df = trade_df[['HSCD', 'AMT', 'ITEM', 'BYRORGNM1', 'SPLYORGNM1', 'ISSUEDT', 'TOTQTY']]
trade_df.head()

,HSCD,AMT,ITEM,BYRORGNM1,SPLYORGNM1,ISSUEDT,TOTQTY
0,7210491090,45360.00,GI COIL,주식회사 포스코대우,POSCO,20180227,565475000.0
1,7210491090,821355.00,GI COIL,주식회사 포스코대우,POSCO,20180227,565475000.0
2,7210491090,398580.00,HG COIL - POSCO,주식회사 포스코대우,POSCO,20180227,565475000.0
3,7225509000,416301.78,CR COIL (0.5MM<=T<1.0MM),주식회사 포스코대우,POSCO,20180227,565475000.0
4,7209169000,8134262.00,CR COIL (3MM<=T),주식회사 포스코대우,POSCO,20180227,565475000.0


In [5]:
len(trade_df['BYRORGNM1'].unique())

16618

In [6]:
len(trade_df['HSCD'].unique())

7252

In [4]:
hscode_trade_table = trade_df.groupby(['HSCD', 'BYRORGNM1', 'SPLYORGNM1'])['AMT'].sum().reset_index()
hscode_trade_table.head()

,HSCD,BYRORGNM1,SPLYORGNM1,AMT
0,000000000,현대위아(주),(주)위테크,5.700000e+08
1,0000000000,LG전자 (주),백림화학(주),6.753112e+07
2,0000000000,LG전자 (주),성철사(주),4.347540e+07
3,0000000000,LG전자 (주),주)동양이엔피,2.157122e+08
4,0000000000,LG전자 (주),주)엠씨엠,1.312642e+09


In [7]:
hscode_trade_table.loc[hscode_trade_table['HSCD'] == '400911']
# print(' ')

,HSCD,BYRORGNM1,SPLYORGNM1,AMT
28160,400911,현대위아(주),(주)화승R&A,438440.0
28161,400911,현대위아(주),평화산업(주),5049400.0


In [22]:
hscode_trade_table.loc[hscode_trade_table['HSCD'] == '400911']['AMT'] * 100 \
    / hscode_trade_table.loc[hscode_trade_table['HSCD'] == '400911']['AMT'].sum()

28160     7.9893
28161    92.0107
Name: AMT, dtype: float64

In [21]:
# BUYER SHARE
trade_df.groupby(['BYRORGNM1', 'HSCD'])['AMT'].sum().reset_index().head()

,BYRORGNM1,HSCD,AMT
0,YP무역,8519812210,7450000.0
1,영원무역 주식회사,1902301010,343604444.0
2,지니제이 인터내셔널,1704902090,27991.6
3,지니제이 인터내셔널,1905901040,108110.0
4,케이앤씨(KNC),6004100000,19437.0


In [22]:
# SUPPLYER SHARE
trade_df.groupby(['SPLYORGNM1', 'HSCD'])['AMT'].sum().reset_index().head()

,SPLYORGNM1,HSCD,AMT
0,(주)알포메,3921130000,168365.0
1,(주)일호엔지니어링,9031809099,4500.0
2,"sindoh co.,ltd",8443311010,35918131.0
3,삼양발브종합메이커,8481400000,2282566.0
4,에이치알메디텍,8543709090,41080000.0


In [33]:
hscode_trade_table.loc[hscode_trade_table['SPLYORGNM1'] == '(주)알포메']

,HSCD,BYRORGNM1,SPLYORGNM1,AMT
10621,2929101000,흥진수지,(주)알포메,6749356.0
23852,3921130000,(주)세림티티시,(주)알포메,8098465.0


In [25]:
config = yaml.safe_load(open('dart_config.yml'))
db_url = 'mysql://' + config['DB_USER'] + ':' + config['DB_USER_PASSWORD'] + '@' + config['DB_HOST'] \
                            + '/' + config['DB_NAME'] + '?charset=utf8'
engine = create_engine(db_url, encoding='utf-8')
conn = engine.connect()

In [27]:
company_df = pd.read_sql_table(config['DB_TABLE_NAME_1'], con=conn, index_col='id', 
                          columns=['id', 'name', '업종', '상장일', '지역'])
company_df.head()

,name,업종,상장일,지역
id,,,,
1,롯데케미칼,기초 화학물질 제조업,1991-05-30,서울특별시
2,대한유화,기초 화학물질 제조업,1999-08-11,서울특별시
3,이화산업,기타 전문 도매업,1994-04-29,서울특별시
4,디케이앤디,플라스틱제품 제조업,2018-11-20,경기도
5,송원산업,기타 화학제품 제조업,1977-06-25,울산광역시


In [34]:
company_df.loc[company_df['name'] == trade_df.loc[1, 'BYRORGNM1']]

,name,업종,상장일,지역
id,,,,


In [35]:
account_df = pd.read_sql_table(config['DB_TABLE_NAME_2'], con=conn, index_col='id',
                         columns=['id', 'company_id', 'year', 'quarter', 'data_period', 'start_date', 'end_date',
                                 'error', '매출액', '매출원가', '매출총이익', '판매비와관리비', '영업이익', '기타수익',
                                 '기타비용', '지분법이익', '금융수익', '금융비용', '법인세비용차감전순이익', '법인세비용',
                                 '분기순이익'])
account_df.head()

,company_id,year,quarter,data_period,start_date,end_date,error,매출액,매출원가,매출총이익,판매비와관리비,영업이익,기타수익,기타비용,지분법이익,금융수익,금융비용,법인세비용차감전순이익,법인세비용,분기순이익
id,,,,,,,,,,,,,,,,,,,,
1,1,2012,1,quarter,2012-01-01,2012-03-31,None,385.037,354.284,30.75290,10.4784,21.96120,7.92900,NaN,NaN,3.57648,3.55188,22.51700,4.337470,18.17950
2,1,2012,2,quarter,2012-01-01,2012-06-30,None,394.880,388.575,6.30579,11.0771,-2.83395,6.12777,NaN,NaN,1.83494,2.85516,-3.29184,-1.875760,-1.41608
3,1,2012,3,quarter,NaT,NaT,None,413.187,382.764,30.42240,11.0637,20.22220,4.32802,NaN,NaN,2.78657,3.82426,20.31320,-4.577300,15.73590
4,1,2012,4,quarter,2012-01-01,2012-12-31,None,350.719,362.628,-11.90660,8.9963,-23.49350,NaN,NaN,NaN,3.53929,2.76902,-22.25280,19.772700,-15.59320
5,1,2013,1,quarter,2013-01-01,2013-03-31,None,417.124,394.622,22.50180,10.7584,11.74340,NaN,NaN,NaN,4.37646,6.42669,11.73790,0.372595,11.36530
